In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

PATH = "C:\\Program Files (x86)\\chromedriver.exe"

In [16]:
def search(driver, position):
    
    # enter position name in search bar
    search = driver.find_element_by_name("q")
    search.send_keys(position)

    # clear loaction name
    location = driver.find_element_by_id('text-input-where')
    location.send_keys(Keys.CONTROL + "a")
    location.send_keys(Keys.DELETE)

    # execute search
    search.send_keys(Keys.RETURN)

In [101]:
# create drivers, access indeede
drivers = [webdriver.Chrome(PATH) for i in range(2)]
drivers[0].get("https://www.indeed.com/")

# seach for data science, get jobs on page
search(drivers[0], "data science")
jobs = drivers[0].find_elements(By.CLASS_NAME,'result')

# data collection
headers = []
descriptions = []

# iterate over jobs
for job in jobs:
    
    # access individual job container frame
    job.click()
    idek  = drivers[0].find_element(By.ID, "vjs-container-iframe")
    
    # open job container in seperate chrome driver
    drivers[1].get(idek.get_attribute("src"))
    
    # access header and description info
    headers.append(drivers[1].find_element(By.CLASS_NAME, "jobsearch-JobComponent-embeddedHeader").text)
    desciptions.append(drivers[1].find_element(By.ID, "jobDescriptionText").text)

# close drivers
[driver.close() for driver in drivers]
print(f"Found {len(desciptions)} jobs")

Found 45 jobs
